In [1]:
import pandas as pd

In [2]:
nlg = pd.read_csv("../data/recipeNLG/full_dataset.csv")

In [3]:
nlg.head(3)

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."


In [4]:
nlg.source.unique()

array(['Gathered', 'Recipes1M'], dtype=object)

In [5]:
# !pip install nltk
# import nltk
# from nltk.stem import WordNetLemmatizer
# from nltk.tokenize import word_tokenize
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [6]:
nlg["NER"][0]

'["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'

In [7]:
nlg["directions"][0]

'["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]'

## GISMO ingredient preprocessing

In [8]:
from scripts.preprocessing import *


args = Args(minnumingrs=2, minnuminstrs=3, maxnuminstrs=12, maxnumingrs=10, 
            minnumwords=50, majority_lvl=0.5, threshold_ingrs=10)

ingredient_parser = IngredientParser(
    replace_dict={
        "and": ["&", "'n"],
        "": ["%", ",", ".", "#", "[", "]", "!", "?"],
    }
)

instruction_parser = InstructionParser(
    replace_dict={"and": ["&", "'n"], "": ["#", "[", "]"]}
)

ingrs, cluster_ingrs = ingredients_dict(nlg, args, instruction_parser, ingredient_parser)


In [9]:
from itertools import islice

print(dict(islice(cluster_ingrs.items(), 0, 2)))
print(dict(islice(ingrs.items(), 0, 2)))
print(len(cluster_ingrs))

{'macaroni': ['elbow_macaroni', 'macaroni', 'deluxe_macaroni', 'whole_wheat_elbow_macaroni', 'favourite_macaroni', 'shell_macaroni', 'pasta_elbow_macaroni', 'short_macaroni', 'whole_wheat_macaroni', 'salad_cooked_macaroni', 'packagekraft_deluxe_macaroni', 'spiral_macaroni', 'barilla_macaroni', 'ring_macaroni', 'weight_macaroni', 'macaroni_cooked', 'wagon_wheel_macaroni', 'multi-grain_elbow_macaroni', 'elbo_macaroni', 'bowl_of_macaroni', 'salad_macaroni', 'trio_macaroni', 'cooked_shell_macaroni', 'whole_grain_elbow_macaroni', 'orzo_macaroni', 'colored_macaroni', 'boxwhite_cheddar_macaroni', 'fibre-enriched_elbow_macaroni', 'boxvelveeta_macaroni', 'boxof_uncooked_elbow_macaroni', 'baby_shell_macaroni', 'cooled_salad_macaroni', 'acini_macaroni', 'penne_macaroni', 'pasta_shell_macaroni', 'packagekraft_macaroni', 'cooked_spiral_macaroni', 'cork_screw_macaroni', 'velveeta_macaroni', 'seashell_macaroni', 'creamettes_macaroni', 'packagemulti-colored_corkscrew_macaroni', 'multi-colored_macaroni

https://www.kaggle.com/datasets/thedevastator/now-with-more-nutrients/

In [10]:
kaggle = pd.read_csv("../data/cleaned_ingredients.csv")

In [11]:
kaggle.head(3)

,NDB_No,Descrip,Energy_kcal,Protein_g,Saturated_fats_g,Fat_g,Carb_g,Fiber_g,Sugar_g,Calcium_mg,...,VitC_mg,Thiamin_mg,Riboflavin_mg,Niacin_mg,VitB6_mg,Folate_mcg,VitB12_mcg,VitA_mcg,VitE_mg,VitD2_mcg
0,1001,butter with salt,717.0,0.85,51.368,81.11,0.06,0.0,0.06,24.0,...,0.0,0.005,0.034,0.042,0.003,3.0,0.17,684.0,2.32,0.0
1,1002,butter whipped w salt,718.0,0.49,45.390,78.30,2.87,0.0,0.06,23.0,...,0.0,0.007,0.064,0.022,0.008,4.0,0.07,683.0,1.37,0.0
2,1003,butter oil anhydrous,876.0,0.28,61.924,99.48,0.00,0.0,0.00,4.0,...,0.0,0.001,0.005,0.003,0.001,0.0,0.01,840.0,2.8,0.0


In [12]:
import ast
from scripts.ingredient import *

def kaggle_dict(kaggle, ingredient_parser: IngredientParser):
    ingr_proteined = {}
    for name, protein in zip(kaggle['Descrip'], kaggle['Protein_g']):
        protein_f = float(protein)
        # preprocess ingr, 
        ready_ingr = ingredient_parser.parse_entry(name)
        ingr_proteined[ready_ingr] = protein_f

    return ingr_proteined

In [13]:
kaggle_prot = kaggle_dict(kaggle, ingredient_parser)

In [14]:
print(dict(islice(kaggle_prot.items(), 0, 3)))

{'butter_with_salt': 0.85, 'butter_whipped_w_salt': 0.49, 'butter_oil_anhydrous': 0.28}


In [15]:
def coverage_strict(cluster_ingrs, kaggle_prot):
    count = 0
    for ingr in kaggle_prot.keys():
        for possible_names in cluster_ingrs.values():
            if ingr in possible_names:
                count += 1
    return count / len(cluster_ingrs)


def coverage_chill(cluster_ingrs, kaggle_prot):
    count = 0
    used = set()
    for ingr in kaggle_prot.keys():
        first_part_of_ingr = ingr.split(sep="_")[0]
        if first_part_of_ingr in used:
            continue
        else:
            for possible_names in cluster_ingrs.values():
                if first_part_of_ingr in possible_names:
                    count += 1
                used.add(first_part_of_ingr)
    return count / len(cluster_ingrs)


def not_covered(cluster_ingrs, kaggle_prot):
    count = 0
    kaggle_set = set()
    not_covered = set()
    for ingr in kaggle_prot.keys():
        first_part_of_ingr = ingr.split(sep="_")[0]
        kaggle_set.add(first_part_of_ingr)
    for ingr in cluster_ingrs.keys():
        if ingr not in kaggle_set:
            not_covered.add(ingr)
    return not_covered

In [16]:
print(coverage_strict(cluster_ingrs, kaggle_prot))
print(coverage_chill(cluster_ingrs, kaggle_prot))

0.023048228084199308
0.07633892885691447


In [17]:
not_cov = not_covered(cluster_ingrs, kaggle_prot)
print(not_cov)

{'hardbread', 'below', 'appreciation', 'tostados', 'bakingsoda', 'vegall', 'albarino', 'jarrobusto', 'imperial', 'casseroles', 'medium_sized', 'temp', 'trays', 'ramona_wachter', 'bottlehiram_walker_original_cinn', 'ligurian', 'tomalley', 'velveta', 'ketch', 'charms', 'circle', 'f', 'red-pepper', 'perrier', 'scorpion', 'coconut-pecan', 'mousse', 'cheerios', 'hellmanns', 'sauvignon', 'whipcream', 'scotch', 'carl_budding', 'brandy-soaked', 'vermont', 'folgers', 'ombine', 'tamarai', 'navan', 'barlette', 'bulky', 'marjorin', 'vienna', 'jamon', 'canlucky', "avocado's", 'broil', 'joyu', 'witlof', 'slider', 'orcebollitas_asadas', 'basics', 'jarlsberg', 'habenero', 'cafe_louisianne', 'orange-flower', 'cavatelli', 'besan', 'chester', 'syrofoam', 'chutney', 'guinness', 'colby-monterey', 'whippi', 'crimini', 'tentsuyu', 'lattice', 'zafrig', 'ladleful', 'chronicles', 'potates', 'nonalcoholic', 'thaw', 'whtie_miniature', 'guyre', 'room', 'grated_red_leichester', 'cartons', 'enveople', 'kintsay', 'va